## Train the Deep Learning Model on the Intracranial Hemorrhage Dataset
***

The model available for fine-tuning attaches a classification layer to the corresponding feature extractor model available on TensorFlow/PyTorch hub, and initializes the layer parameters to random values. The output dimension of the classification layer is determined based on the number of classes in the input data. The fine-tuning step fine-tunes the model parameters. The objective is to minimize classification error on the input data. The model returned by fine-tuning can be further deployed for inference. Below are the instructions for how the training data should be formatted for input to the model. 

- **Input:** A directory with as many sub-directories as the number of classes. 
    - Each sub-directory should have images belonging to that class in .jpg format. 
- **Output:** A trained model that can be deployed for inference. 
    - A label mapping file is saved along with the trained model file on the s3 bucket.  
    
The input directory should look like below if 
the training data contains images from two classes: roses and dandelion. The s3 path should look like
`s3://bucket_name/input_directory/`. Note the trailing `/` is required. The names of the folders and 'hemorrhage', 'no_hemorrhage', and the .jpg filenames
can be anything. The label mapping file that is saved along with the trained model on the s3 bucket maps the 
folder names 'hemorrhage' and 'hemorrhage' to the indices in the list of class probabilities the model outputs.
The mapping follows alphabetical ordering of the folder names. In the example below, index 0 in the model output list
would correspond to 'dandelion' and index 1 would correspond to 'roses'.

    input_directory
        |--hemorrhage
            |--ID_1b7a626dd.jpg
            |--ID_050464d4c.jpg
        |--no_hemorrhage
            |--ID_0007a3b49.jpg
            |--ID_00096021a.jpg
***

## Set Up
Before executing the notebook, there are some initial steps required for setup. This notebook requires ipywidgets.

In [2]:
!pip install ipywidgets==7.0.0 --quiet


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


To train and host on Amazon Sagemaker, we need to setup and authenticate the use of AWS services. Here, we use the execution role associated with the current notebook instance as the AWS account role with SageMaker access. It has necessary permissions, including access to your data in S3. 

In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## Select a Computer Vision Model and Retrieve Training artifacts

Select the model we'll be using, which is the efficientnet B7 model. A complete list of SageMaker models can also be accessed at [Sagemaker pre-trained Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html#).

Here, for the selected model, we retrieve the training docker container, the training algorithm source, the pre-trained base model, and a python dictionary of the training hyper-parameters that the algorithm accepts with their default values. Note that the model_version="*" fetches the lates model. Also, we do need to specify the training_instance_type to fetch train_image_uri.


In [4]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = "pytorch-ic-densenet121", "*"
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

### Set Training parameters
Now that we are done with all the setup that is needed, we are ready to fine-tune our Image Classification model. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator will launch the training job. 
There are two kinds of parameters that need to be set for training. 

The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training. We defined the training instance type above to fetch the correct train_image_uri. 

The second set of parameters are algorithm specific training hyper-parameters.


In [5]:
# Sample training data is available in this bucket
training_data_bucket = "re-invent-radiology-demo"
training_data_prefix = "radiology_image_data_model/input_training_data/"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = "re-invent-radiology-demo"
output_prefix = "radiology_image_data_model/output/"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"


***
For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.
***

In [6]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparameters["epochs"] = "5"
hyperparameters["augmentation"] = "True"
hyperparameters["train_only_top_layer"] = "False"
hyperparameters["batch_size"] = "32"

print(hyperparameters)

{'train_only_top_layer': 'False', 'epochs': '5', 'learning_rate': '0.001', 'batch_size': '32', 'reinitialize_top_layer': 'Auto', 'augmentation': 'True'}


### Train with Automatic Model Tuning ([HPO](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)) <a id='AMT'></a>
***
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We will use a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) object to interact with Amazon SageMaker hyperparameter tuning APIs.
***

In [7]:
from sagemaker.tuner import ContinuousParameter

# Use AMT for tuning and selecting the best model
use_amt = False

# Define objective metric per framework, based on which the best model will be selected.
metric_definitions_per_model = {
    "tensorflow": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
    "pytorch": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val Acc: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 1

### Start Training
We start by creating the estimator object with all the required assets and then launch the training job.


In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"radiology-hemorrhage-{model_id}-transfer-learning")

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

if use_amt:
    metric_definitions = next(
        value for key, value in metric_definitions_per_model.items() if model_id.startswith(key)
    )

    hp_tuner = HyperparameterTuner(
        ic_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

INFO:sagemaker:Creating training-job with name: radiology-hemorrhage-pytorch-ic-densene-2023-10-26-22-07-46-647


2023-10-26 22:07:46 Starting - Starting the training job...
2023-10-26 22:08:11 Starting - Preparing the instances for training.........
2023-10-26 22:09:31 Downloading - Downloading input data................................................
2023-10-26 22:17:48 Training - Downloading the training image...
2023-10-26 22:18:13 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-26 22:18:27,987 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-26 22:18:28,020 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-26 22:18:28,025 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-10-26 22:18:28,392 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs

## Deploy & run Inference on the model

A trained model does nothing on its own. We now want to use the model to perform inference. We start by retrieving the artifacts for deploying an endpoint.


In [10]:
inference_instance_type = "ml.p2.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"radiology-hemorrhage-FT-{model_id}-")

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = (hp_tuner if use_amt else ic_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

INFO:sagemaker:Creating model with name: sagemaker-jumpstart-2023-10-27-03-25-48-201
INFO:sagemaker:Creating endpoint-config with name radiology-hemorrhage-FT-pytorch-ic-dens-2023-10-27-03-25-48-201
INFO:sagemaker:Creating endpoint with name radiology-hemorrhage-FT-pytorch-ic-dens-2023-10-27-03-25-48-201


-----------------!

In [11]:
#create a list with the path for a few sample images
test_images = ['<test image 1 path>', '<test image 2 path>']

Next, we query the model, parse the response and display the predictions.

In [12]:
import pandas as pd
from IPython.core.display import HTML
combined_df = pd.DataFrame()

for image_filename in test_images:
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    preds = {key: [value for value in values] for key, values in model_predictions.items() if key != 'predicted_label'}
    df = pd.DataFrame(preds)
    df = df[df['probabilities']==df['probabilities'].max()].reset_index(drop = True)
    combined_df = pd.concat([combined_df, df], ignore_index = True)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )

In [ ]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()